In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install tiktoken
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

In [ ]:
# Import necessary libraries
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Configure the baseline configuration of the OpenAI library for Azure OpenAI Service.
OPENAI_API_KEY = "KEY"

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

<ipython-input-4-3f1a2a7039c2>:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=OPENAI_API_KEY)


In [ ]:
llm.invoke("What is EDA?")

'\n\nEDA stands for Exploratory Data Analysis. It is a process of examining and analyzing data sets to discover patterns, trends, and relationships in the data. EDA involves visualizing data through charts, graphs, and other methods to gain insights and understanding about the data. It is often the first step in the data analysis process and helps to identify potential problems or areas of interest for further investigation. EDA is commonly used in data science, statistics, and other fields to understand and extract valuable information from data sets.'

#### using **custom** dataset

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 3.3 MB/s eta 0:00:00


In [ ]:
pdf_reader = PyPDFLoader("/content/tcs.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, )
chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Create embeddings
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
db = FAISS.from_documents(documents=chunks, embedding=embeddings)


<ipython-input-10-f733b0a55cd8>:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [ ]:
chat_history = []
query = """Does the vendor have experience with similar industries and use cases?​?​​"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

 Yes, the vendor has experience with similar industries and use cases as they have previously worked with a large US manufacturing client and are currently working with a large manufacturing North American client. 


In [ ]:
chat_history = []
query = """?Does the vendor's financial offer make sense compared to the timeline, resources, and deliverables??"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

 Without more specific information about the vendor's financial offer, it is impossible to accurately determine if it makes sense compared to the timeline, resources, and deliverables. 


In [ ]:
chat_history = []
query = """?What is RAGs?"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

 RAGs stands for Retriever-Reader for Open-Domain Question Answering. It is a model that uses external knowledge sources, such as Wikipedia, to generate accurate answers to open-domain questions. RAGs have shown to outperform other models in terms of accuracy and diversity of generated answers.
